In [71]:
import requests
import json
import csv

def read_cnv_file(file_path):
    cnv_data = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(',') 
            if len(parts) == 3: 
                date, company, news_name = parts
                cnv_data.append((date, company, news_name))
    return cnv_data

file_path = 'NewsTesla.csv'
cnv_data = read_cnv_file(file_path)
url = 'http://localhost:11434/api/generate'

with open('Toutput.csv', 'w', newline='') as csvfile:
    fieldnames = ['Date', 'News Name', 'Confidence', 'Advice']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for idx, (date, company, news_name) in enumerate(cnv_data):
        if idx == 0:
            continue
        
        news = f"{news_name}"
        data = {
            "model": "my-llama3",
            "prompt": news,
            "format": "json",
            "stream": False
        }
        response = requests.post(url, json=data)
        if response.status_code == 200:
            try:
                result = response.json()
                stock_info = result['response'].strip()
                stock_infoDict = json.loads(stock_info)
                print(stock_info_dict)
                stock_info_lower = {k.lower(): v for k, v in stock_infoDict.items()}
                confidence = stock_info_lower.get('confidence')
                advice = stock_info_lower.get('advice')
                if confidence is not None and advice is not None:
                    print(f"Confidence: {confidence}, Advice: {advice}")
                    writer.writerow({'Date': date, 'News Name': news_name, 'Confidence': confidence, 'Advice': advice})
                else:
                    print("ERROR datas have not the same structure")
            except (KeyError, ValueError) as e:
                print("error in json:", e)
        else:
            print("ERRO in respons with status code ", response.status_code)

{'name': 'Tesla Energy Storage Product', 'confidence': 80, 'advice': 'Buy'}
Confidence: 80, Advice: Buy
{'name': 'Tesla', 'confidence': 80, 'advice': 'Hold'}
Confidence: 80, Advice: Hold
{'stock': 'TSLA', 'confidence': 60, 'advice': 'HOLD'}
Confidence: 60, Advice: HOLD
{'stock': 'TSLA', 'confidence': 80, 'advice': 'Buy'}
Confidence: 80, Advice: Buy
{'name': 'Tesla', 'confidence': 50, 'advice': 'Hold'}
Confidence: 50, Advice: Hold
{'name': 'Tesla', 'confidence': 70, 'advice': 'Sell'}
Confidence: 70, Advice: Sell
{'name': 'Tesla', 'confidence': 80, 'advice': 'Buy'}
Confidence: 80, Advice: Buy
